To do:
- Have a look at learning rate and gradient norm clipping which I need to read up on.
    - Setting learning rate to 1e-4 from the "Embedding sweep" section of the mmBERT paper
    - Keeping gradient norm clipping to the default which caps it at 1.0

- Hyperparameter tuning (Alpha, learning rate, batch size so on - not sure how to figure this out)
    - There is precedence for no hyperparameter tuning from the author of the OG NLI model that DEBATE is based on = Due to computational restrains and the points from this paper, no hyperparameter tuning was performed in this case. The model tuning in itself is also not the primary focus in this paper, but simply serves as a tool for the actual inquiry into blame in the Danish Parliament



In [ ]:
%pip install -r "requirements_bert.txt"

In [1]:
import torch
import transformers
import bitsandbytes
import accelerate
import datasets
import numpy as np
import pandas as pd
import keras
import json
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from sklearn.model_selection import train_test_split
from datasets import Dataset
from keras.losses import binary_crossentropy
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, recall_score

/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-24 15:06:21.858325: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-24 15:06:21.907398: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-24 15:06:22.884977: I tensorflow/core/util/port.cc:153] oneDNN custom 

In [2]:
model_name = "jhu-clsp/mmBERT-base"

quantization_config = BitsAndBytesConfig(
                                        load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16,
                                         bnb_4bit_quant_type="nf4",
                                         bnb_4bit_use_double_quant=True,
                                         )

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
    dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at jhu-clsp/mmBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
lora_config = LoraConfig(
    r=16,  # Low-rank dimension
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules="all-linear",  # Fine-tuning all linear (classification, attention... layers)
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,416,096 || all params: 310,947,874 || trainable%: 1.0986


In [4]:
training_args = TrainingArguments(
    report_to='wandb',
    output_dir='./full_tune_results',
    optim="paged_adamw_8bit",
    learning_rate=1e-4, # Learning rate copied from mmBERT paper on embedding sweep of LR (1e-4) as they found this to perform best
    num_train_epochs=3,
    per_device_train_batch_size=256, # Batching at 256 to balance generalization and efficient training
    gradient_accumulation_steps=1,  # Gradient of 1 as full batch fits in memory, accumulation then only slows
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    bf16=True,  # Enable mixed precision
    fp16=False,
    dataloader_pin_memory=True,
    dataloader_num_workers=8,
    remove_unused_columns=True, # Avoiding manual handling of residual text columns
    max_grad_norm=1.0,
    disable_tqdm=False,
)

In [18]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
    padding="max_length",
    truncation=True,
    max_length=512, # Padding to 512 to massively cut down on computation compared to base 8,192 tokens. 
    )

In [6]:
def weighted_bincrossentropy(true, pred, weight_zero = 99.0, weight_one = 1):
    """
    Calculates weighted binary cross entropy. The weights are fixed to represent class imbalance in the dataset.
        
    For example if there are 10x as many positive classes as negative classes,
        if you adjust weight_zero = 1.0, weight_one = 0.1, then false positives 
        will be penalized 10 times as much as false negatives.

    """
  
    # calculate the binary cross entropy
    bin_crossentropy = binary_crossentropy(true, pred)
    
    # apply the weights
    weights = true * weight_one + (1. - true) * weight_zero
    #weights /= (weight_one + weight_zero) # Normalizing to be more consistent with regular BCE for comparison 
    weighted_bin_crossentropy = weights * bin_crossentropy 

    return np.mean(weighted_bin_crossentropy)

In [7]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    #From logits to probabilities
    probs_2d = np.exp(predictions) / np.exp(predictions).sum(axis=1, keepdims=True)
    probs = probs_2d[:, 1]  # positive class extraction
    
    weigthted_bce = weighted_bincrossentropy(labels, probs)
    keras_bce = binary_crossentropy(labels, probs)
    keras_bce = float(np.mean(keras_bce.numpy()))  # Converting from keras eagertensor to float value
    
    # Wrapping all metrics to floats for json serialization during model eval
    return {
        'keras_BCE': keras_bce,
        'weighted BCE': weigthted_bce,
        'recall': float(recall_score(labels, probs.round())),
        'precision': float(average_precision_score(labels, probs)),
        'accuracy': float(accuracy_score(labels, probs.round())), # Need rounding for these two computations (integer required)
        'f1': float(f1_score(labels, probs.round(), average='macro')) # macro f1 is better for imbalanced dataset
    }

In [21]:
# Custom trainer class (weigthed)
from collections import Counter

labels = dataframe['label'].tolist()
class_counts = Counter(labels)
total = sum(class_counts.values())

# Higher weight = more emphasis
weights = [total/class_counts[0], total/class_counts[1]]
class_weights = torch.tensor(weights, dtype=torch.float)

#define custom trainer that uses weigted loss
import torch.nn as nn

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # Define weighted loss
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

In [19]:
val_dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/Model_data/validation_set.json")

val_dataframe = val_dataframe[['text', 'label']]

val_dataset = Dataset.from_pandas(val_dataframe)

tokenized_val = val_dataset.map(tokenize_function, batched=True, num_proc=16)

Map (num_proc=16): 100%|██████████| 258/258 [00:01<00:00, 226.09 examples/s]


In [ ]:
test_dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/training_data/cleaned_training_data.json")

test_dataframe = test_dataframe[['text', 'label']]

test_dataframe = test_dataframe[0:10000]

test_dataset = Dataset.from_pandas(test_dataframe)

tokenized_test = test_dataset.map(tokenize_function, batched=True, num_proc=16)

Map: 100%|██████████| 10000/10000 [00:01<00:00, 5292.08 examples/s]


In [20]:
dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/training_data/preproc_data_for_tuning_final.json")

dataframe = dataframe[['text', 'label']]

dataset = Dataset.from_pandas(dataframe)

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=16)

Map (num_proc=16): 100%|██████████| 388027/388027 [00:15<00:00, 25190.14 examples/s]


In [22]:
'''
Look into learning rates, model is currently overfitting quite drastically ("small" test-set)
Normalizing weigthed BCE or no?
Look into regularization, dropout and early stopping to avoid overfitting
'''

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)


trainer.train()

/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  ret

Epoch,Training Loss,Validation Loss,Keras Bce,Weighted bce,Recall,Precision,Accuracy,F1
1,0.081200,0.428823,0.416334,27.300520,0.829545,0.808965,0.852713,0.839510
2,0.086800,0.531797,0.432498,28.360483,0.795455,0.861925,0.864341,0.848680
3,0.171600,1.194470,0.628947,41.242348,0.579545,0.823916,0.802326,0.763085


/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
W0000 00:00:1761318513.848598    2166 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarni

TrainOutput(global_step=4548, training_loss=0.28072537823433585, metrics={'train_runtime': 20756.6383, 'train_samples_per_second': 56.082, 'train_steps_per_second': 0.219, 'total_flos': 4.088858875877192e+17, 'train_loss': 0.28072537823433585, 'epoch': 3.0})

In [ ]:
# Load the model from the specific checkpoint
best_model = AutoModelForSequenceClassification.from_pretrained("./full_tune_results/checkpoint-3032")



# Save permanently to a new folder
best_model.save_pretrained("/work/RuneEgeskovTrust#9638/Bachelor/best_model_epoch2")
tokenizer.save_pretrained("/work/RuneEgeskovTrust#9638/Bachelor/best_model_epoch2")

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at jhu-clsp/mmBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('/work/RuneEgeskovTrust#9638/Bachelor/best_model_epoch2/tokenizer_config.json',
 '/work/RuneEgeskovTrust#9638/Bachelor/best_model_epoch2/special_tokens_map.json',
 '/work/RuneEgeskovTrust#9638/Bachelor/best_model_epoch2/tokenizer.json')

In [ ]:
from peft import PeftModel
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 1️⃣ Load the base model
base_model_name = "jhu-clsp/mmBERT-base"  # or whatever your base model is
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name)

# 2️⃣ Load the LoRA adapter from the 2nd-epoch checkpoint
adapter_checkpoint = "./full_tune_results/checkpoint-3032"
lora_model = PeftModel.from_pretrained(base_model, adapter_checkpoint)

# 3️⃣ Merge LoRA weights into the base model
merged_model = lora_model.merge_and_unload()

# 4️⃣ Save the merged model as a standalone Hugging Face model
save_dir = "/work/RuneEgeskovTrust#9638/Bachelor/merged_model_epoch2"
merged_model.save_pretrained(save_dir)

'''
# 5️⃣ Save tokenizer too
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
'''


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at jhu-clsp/mmBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'\n# 5️⃣ Save tokenizer too\ntokenizer = AutoTokenizer.from_pretrained(base_model_name)\ntokenizer.save_pretrained(save_dir)\n'

In [32]:
trainable_params = [p for p in merged_model.parameters() if p.requires_grad]
print(f"Trainable parameters after merge: {len(trainable_params)}")


Trainable parameters after merge: 0


In [28]:
tokenizer.save_pretrained(save_dir)


('/work/RuneEgeskovTrust#9638/Bachelor/merged_model_epoch2/tokenizer_config.json',
 '/work/RuneEgeskovTrust#9638/Bachelor/merged_model_epoch2/special_tokens_map.json',
 '/work/RuneEgeskovTrust#9638/Bachelor/merged_model_epoch2/tokenizer.json')

In [23]:
FINE_TUNED_MODEL_NAME = "mmBlameBERT-pol-DA"

#merged_model = model.merge_and_unload()    # PEFT: incorporates LoRA into base weights
merged_dir = f"/work/RuneEgeskovTrust#9638/Bachelor/{FINE_TUNED_MODEL_NAME}-final-tune"
merged_model.save_pretrained(merged_dir)
#tokenizer.save_pretrained(merged_dir)
print("✓ Merged model saved to:", merged_dir)

NameError: name 'merged_model' is not defined

In [ ]:
trainer.save_model("/work/RuneEgeskovTrust#9638/Bachelor/LORA-adapters-for-final-tune")

In [29]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 1.1944702863693237, 'eval_keras_BCE': 0.6289470195770264, 'eval_weighted BCE': 41.242347717285156, 'eval_recall': 0.5795454545454546, 'eval_precision': 0.8239160634974602, 'eval_accuracy': 0.8023255813953488, 'eval_f1': 0.7630853994490359, 'eval_runtime': 1.9366, 'eval_samples_per_second': 133.222, 'eval_steps_per_second': 17.04, 'epoch': 3.0}


In [ ]:

with open("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/Final_tune_eval_result.txt", "w") as f:
    f.write(str(eval_results))

In [10]:
def preprocess_json(input_path, output_path=None):
    """
    Preprocesses a JSON file by filtering out entries based on the 'text' key.
    
    Criteria for deletion:
      - 'text' is missing or empty
      - 'text' length is <= 3
      - 'text' contains '(' or ')'
    
    Parameters:
        input_path (str): Path to the input JSON file.
        output_path (str, optional): If provided, saves the filtered JSON here.
    
    Returns:
        list: The filtered list of JSON entries.
    """
    # Load JSON file
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Filter entries
    filtered_data = [
        entry for entry in data
        if 'text' in entry
        and entry['text']
        and len(entry['text']) > 3
        and '(' not in entry['text']
        and ')' not in entry['text']
    ]

    # Optionally save to a new file
    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(filtered_data, f, ensure_ascii=False, indent=4)

    return filtered_data

In [ ]:
del model
torch.cuda.empty_cache()

In [12]:
preprocess_json("/work/RuneEgeskovTrust#9638/Bachelor/training_data/cleaned_training_data_3_4_5_temps.json",
"/work/RuneEgeskovTrust#9638/Bachelor/training_data/preproc_data_for_tuning_final.json")

[{'paragraph': '2',
  'sentence_nr': '235',
  'text': 'Regeringen vil også fortsætte sin offensive  miljøpolitik.',
  'speaker': 'Poul Nyrup Rasmussen',
  'party': 'S',
  'preceding_sentence': 'Der indføres en belønningsordning for virksomheder, der gør  en særlig ekstraindsats ud over det, loven stiller krav om,  for at forbedre arbejdsmiljøet.',
  'succeeding_sent': 'Det er vores næste store felt.',
  'current_speaker_in_government': True,
  'parties_in_government': ['S', 'RV'],
  'date': '1997-10-07 00:00:00',
  'label': 1},
 {'paragraph': '7',
  'sentence_nr': '3',
  'text': 'Sådan som debatten var foregået, sagde hr. Torben Lund,  var det lavt og nedrigt og pinligt.',
  'speaker': 'Pia Kjærsgaard',
  'party': 'DF',
  'preceding_sentence': 'En løftet pegefinger fra den  socialdemokratiske ordfører!',
  'succeeding_sent': 'Jeg vil tillade mig at  sige, at det er lavt og nedrigt og pinligt af den siddende  regering, at den overhovedet ikke har taget debatten op før  nu, når der er et